In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import TCNModel
from darts.dataprocessing.transformers import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from darts.utils.likelihood_models import QuantileRegression
import sys
import json
sys.path.append('../Helper/')
from dataPreprocessing import get_untransformed_exog, TRAIN_DATA_PATH_1990S, TEST_DATA_PATH_1990S
from PyTorchModular import darts_optuna,save_model_hyper_params, train_valid_split_darts, load_prediction
from torch.nn import MSELoss
from torch.optim import AdamW
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import numpy as np
import os

In [2]:
target_col = 'fred_PCEPI'
date_col = 'observation_date'

In [3]:
def get_exog_target(path : str, date_col : str, target_col : str):
    data = pd.read_csv(path)
    data= get_untransformed_exog(data)
    df= data.copy()
    df[date_col] = pd.to_datetime(df[date_col], format='%m/%Y')
    df.set_index(date_col, inplace=True)
    return df.drop(target_col,axis=1), df[target_col]

In [4]:
train_exog, train_target = get_exog_target(TRAIN_DATA_PATH_1990S, date_col, target_col)

test_exog, test_target = get_exog_target(TEST_DATA_PATH_1990S, date_col, target_col)

In [5]:
N_TRIALS=100

In [6]:
search_space = {
    'kernel_size': (int, 1, 6),
    'num_filters': (int, 1, 10),
    'num_layers': (int, 1, 10),
    'dilation_base': (int, 1, 5),
    'weight_norm': (str, [False, True]),
    'dropout': (float, 0.0, 1.0),
    'optimizer_kwargs': {"lr": (float, 1e-5, 1e-1,{'step':None,'log':True})},
    'loss_fn':('categorical',["QuantileRegression", "MSE"])
}

In [7]:
def load_TCN_params(json_path):

    with open(json_path,'r') as f:
        loaded_params= json.load(f)

    if 'loss_fn' in loaded_params.keys():
        if loaded_params['loss_fn']=='MSE':
            loaded_params['loss_fn']=MSELoss()
            loaded_params['likelihood']=None
        else:
            loaded_params['loss_fn']=None
            loaded_params['likelihood']=QuantileRegression((0.25,0.5,0.75))

    loaded_params['optimizer_kwargs']={"lr":loaded_params["lr"]}
    del loaded_params["lr"]

    return loaded_params


def train_TCN(best_params_path, valid_size,horizon):
    
    best_params= load_TCN_params(best_params_path)
    target_scaler = Scaler()
    exog_scaler = Scaler()

    probabilistic= True if best_params['loss_fn'] is None else False

    train_target, valid_target, train_exog, valid_exog=train_valid_split_darts(train_exog,train_target,valid_size,best_params['input_chunk_length'])

    train_target= target_scaler.fit_transform(train_target)
    train_exog= exog_scaler.fit_transform(train_exog)

    valid_target= target_scaler.transform(valid_target)
    valid_exog= exog_scaler.transform(valid_exog)

    early_stopper = EarlyStopping(
                monitor="val_loss",
                patience=25,
                min_delta=1e-5,
                mode="min",
                verbose=VERBOSE
                )
    best_params["save_checkpoints"]=True
    best_params["force_reset"]=True
    best_params["random_state"]=42
    best_params["output_chunk_length"]=horizon
    best_params['optimizer_cls']=AdamW
    best_params['add_encoders']={
        "cyclic": {"future": ["month", "quarter"]}
        }

    best_params["pl_trainer_kwargs"]={
                "accelerator":'auto',
                "callbacks": [early_stopper],
                "verbose": VERBOSE
            }

    model = TCNModel(**best_params, model_name= f'TCN_horizon_{horizon}')

    model.fit(train_target,
            past_covariates=train_exog,
            val_series=valid_target,
            val_past_covariates=valid_exog,
            epochs=10000,
            verbose=VERBOSE)

    model.load_from_checkpoint(model_name=f'TCN_horizon_{horizon}', best=True)

    return model, target_scaler, exog_scaler, best_params['input_chunk_length'], probabilistic

def infer_darts(model,train_exog_df, train_target_series, test_exog_df, test_target_series,exog_scaler,target_scaler, input_size, horizon, probabilistic=False):
    
    combined_exog_df= pd.concat((train_exog_df,test_exog_df),axis=0)
    combined_target_series= pd.concat((train_target_series,test_target_series),axis=0)

    preds_list=np.array([])

    for i in range(0,12,horizon):
        x,y=load_prediction(input_size,combined_exog_df,combined_target_series,exog_scaler,target_scaler,test_target.index[i])
        if probabilistic:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False, num_samples=1000)).values().flatten()
        else:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False)).values().flatten()

        print(test_target.index[i:i+horizon])
        print(pred)
        preds_list= np.append(preds_list,pred)
    
    return preds_list

In [8]:
def TNC_tune_and_save_horizons(horizons, n_trials, verbose=False):
    for horizon in horizons:
        valid_size=horizon+3
        invariates= {
            'output_chunk_length':horizon,
            'optimizer_cls':AdamW,
            'add_encoders':{
                "cyclic": {"future": ["month", "quarter"]}
                }
        }
        # Assumes TNC search space and adpats depending on horizon
        search_space['input_chunk_length'] = (int, 7 if horizon < 7 else horizon+1, 36,{'step':1,'log':False})
        best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,valid_size,horizon,n_trials=n_trials,patience=5,tol=1e-4,verbose=verbose)
        save_model_hyper_params(os.path.join('.', 'bestHyperparameters', f'best_params_tcn_horizon_{horizon}.json'), best_params)

## Horizon = 1

In [9]:
VERBOSE = False

In [ ]:
HORIZON=1
VALID_SIZE=3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

In [ ]:
best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params(os.path.join('.', 'bestHyperparameters', 'best_params_tcn_horizon_1.json'), best_params)

### Horizon = 3

In [ ]:
HORIZON=3
VALID_SIZE=3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

In [ ]:
best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params(os.path.join('.', 'bestHyperparameters', f'best_params_tcn_horizon_{HORIZON}.json'), best_params)

### Horizon = 6

In [ ]:
HORIZON=6
VALID_SIZE=3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

In [ ]:
best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params(os.path.join('.', 'bestHyperparameters', f'best_params_tcn_horizon_{HORIZON}.json'), best_params)

In [ ]:
horizons = [1, 3, 6, 12]

In [ ]:
for HORIZON in horizons:
    VALID_SIZE=HORIZON+3
    invariates= {
        'output_chunk_length':HORIZON,
        'optimizer_cls':AdamW,
        'add_encoders':{
            "cyclic": {"future": ["month", "quarter"]}
            }
    }
    search_space['input_chunk_length'] = (int, 7 if HORIZON < 7 else HORIZON+1, 36,{'step':1,'log':False})
    best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
    save_model_hyper_params(os.path.join('.', 'bestHyperparameters', f'best_params_tcn_horizon_{HORIZON}.json'), best_params)

### Annual Horizon

In [ ]:
HORIZON=12
VALID_SIZE=HORIZON+3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}
search_space['input_chunk_length'] = (int, 7 if HORIZON < 7 else HORIZON+1, 36,{'step':1,'log':False})

In [ ]:
best_params=darts_optuna(TCNModel,'TCN',search_space,invariates,train_target,train_exog,VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params(os.path.join('.', 'bestHyperparameters', f'best_params_tcn_horizon_{HORIZON}.json'), best_params)

In [10]:
TNC_tune_and_save_horizons([12], N_TRIALS)

[I 2025-04-26 21:01:54,404] A new study created in memory with name: TCN_hyperparameter_optimisation
2025-04-26 21:01:54,506 - WARNING - Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
2025-04-26 21:01:54,508 - INFO - Train dataset contains 361 samples.
2025-04-26 21:01:54,514 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025-04-26 21:01:55,788 - INFO - loading best-epoch=2-val_loss=0.01.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: T